## Exercise Question 1
Find the 10 countries with most projects

Import the necessary packages.


In [2]:
#import the necessary packages
import pandas as pd
import json
from pandas.io.json import json_normalize

Next lets load up the file and take a look at the head.

In [21]:
#load json as string
json.load((open('world_bank_projects.json')))

#read in the file as world_bank_data, load it into a DataFrame, then preview the column names
world_bank_data = pd.read_json('world_bank_projects.json')
df = pd.DataFrame(world_bank_data)
df.columns.tolist()

['_id',
 'approvalfy',
 'board_approval_month',
 'boardapprovaldate',
 'borrower',
 'closingdate',
 'country_namecode',
 'countrycode',
 'countryname',
 'countryshortname',
 'docty',
 'envassesmentcategorycode',
 'grantamt',
 'ibrdcommamt',
 'id',
 'idacommamt',
 'impagency',
 'lendinginstr',
 'lendinginstrtype',
 'lendprojectcost',
 'majorsector_percent',
 'mjsector_namecode',
 'mjtheme',
 'mjtheme_namecode',
 'mjthemecode',
 'prodline',
 'prodlinetext',
 'productlinetype',
 'project_abstract',
 'project_name',
 'projectdocs',
 'projectfinancialtype',
 'projectstatusdisplay',
 'regionname',
 'sector',
 'sector1',
 'sector2',
 'sector3',
 'sector4',
 'sector_namecode',
 'sectorcode',
 'source',
 'status',
 'supplementprojectflg',
 'theme1',
 'theme_namecode',
 'themecode',
 'totalamt',
 'totalcommamt',
 'url']

Next lets make a count of each country that appears in the dataframe.

In [4]:
#world_bank_data is loaded as variable df. Using value_counts, lets see the top ten entries are in countries
df['countryname'].value_counts().nlargest(10)

Republic of Indonesia              19
People's Republic of China         19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Nepal                              12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64

As we can see, the top 10 countries with the most projects are:
Republic of Indonesia : 19,
People's Republic of China : 19,
Socialist Republic of Vietnam : 17,
Republic of India : 16,
Republic of Yemen : 13,
People's Republic of Bangladesh : 12,
Nepal : 12,
Kingdom of Morocco : 12,
Republic of Mozambique : 11,
Africa : 11.

## Exercise Question 2
Find the top 10 major project themes (using column 'mjtheme_namecode')

We can use the same technique as the above question to make a count of appearences of entries in the 'mjtheme_namecode column', but this time we will need to use json_normalize to extract values of dictionaries nested in each entry.

In [22]:
#Using to_dict, store each entry as a value with the entry id as the key. orient='records' gives it the format
#[{column -> value}, ... , {column -> value}]. This format is necessary so that json_normalize can access the nested values.
data = df[['id', 'mjtheme_namecode']].to_dict(orient='records')

#use json_normalize to create a new DataFrame based on the contents of the mjtheme_namecode dictionary values
new_df = json_normalize(data, 'mjtheme_namecode')

#Using value_counts, lets see the top ten entries are in 'name'
new_df['name'].value_counts().nlargest(10)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

As we can see, the top 10 project themes are:
Environment and natural resources management    250,
Rural development                               216,
Human development                               210,
Public sector governance                        199,
Social protection and risk management           168,
Financial and private sector development        146,
Social dev/gender/inclusion                     130,
Trade and integration                            77,
Urban development                                50,
and Economic management                              38.

Note that there are 122 entries were 'name' is blank.

## Exercise Question 3
In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [23]:
#initialize a dictionary to assign project theme to code as a value. 
code_to_name = {}

#loop over entries in world_bank_data to create code_to_name dict. Each entry is a list with dictionaries nested within. 
for entry in world_bank_data['mjtheme_namecode']:
    for dic in entry:
    #avoid assigning empty values to keys by specifying dic['name] != ''
        if dic['code'] not in code_to_name.keys() and dic['name'] != '':
            code_to_name[dic['code']] = dic['name']

#take a look at the codes and their corresponding names    
code_to_name

{'1': 'Economic management',
 '10': 'Rural development',
 '11': 'Environment and natural resources management',
 '2': 'Public sector governance',
 '3': 'Rule of law',
 '4': 'Financial and private sector development',
 '5': 'Trade and integration',
 '6': 'Social protection and risk management',
 '7': 'Social dev/gender/inclusion',
 '8': 'Human development',
 '9': 'Urban development'}

In [24]:
#now loop over the dataframe and replace the missing values at 'name' according to their code. 
for entry in world_bank_data['mjtheme_namecode']:
    #since there are more than 1 dictionary with 'name' key, we need to loop over each dictionary as well
    for dic in entry:
        if dic['name'] == '':
            dic['name'] = code_to_name[dic['code']]

#verify that missing names have been replaced
for entry in world_bank_data['mjtheme_namecode']:
    for dic in entry:
        assert (dic['name'] != '')

Our assert throws no errors, verifying that for every entry (a list of dictionaries) in the mjtheme_namecode column, the first dictionary's value at key 'name' is no longer a blank string and has been filled with names associated with the country code key from our code_to_name dictionary. lets test it further by checking the count of each name to make sure there are no ' ' values.

In [17]:
#Again, using to_dict, we store each entry as a value with the entry id as the key.
new_data = world_bank_data[['id', 'mjtheme_namecode']].to_dict(orient='records')

#use json_normalize to create a new DataFrame based on the contents of the mjtheme_namecode dictionary values
new_data = json_normalize(data, 'mjtheme_namecode')

#Using value_counts, lets see the number of entries.
new_data['name'].value_counts()

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Rule of law                                      15
Name: name, dtype: int64

Looking good!  all 1499 entries are accounted for. 